In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import re

from spectrometer import FID1D, plot, process

In [ ]:
# Load raw *.fid data
fids: list[FID1D] = []
for file in Path("../data/some_folder/").glob("*.fid"):
    fids.append(FID1D.from_file(file))

# Extract pulse lengths from metadata
pattern = r"length=(.*)us,delay="
pulse_lengths_us = [float(re.search(pattern, str(fid.pulse))) for fid in fids]

In [ ]:
# Process
signal_strengths = []
for i, fid in enumerate(fids):
    x, y = fid.simple_fft(phase_shift_kwargs=False)
    y2 = np.abs(y) ** 2
    integral = np.trapz(x=x, y=y2)
    signal_strengths.append(integral)

In [ ]:
# Sort data by pulse lengths
pulse_lengths_us, signal_strengths = zip(
    *sorted(zip(pulse_lengths_us, signal_strengths))
)

# Plot raw data
cm = 1 / 2.54
fig, axes = plt.subplots(
    figsize=(16.0 * cm, 8.0 * cm),  # Wide Layout: ~16cm, Margin Layout: 10.7cm
    layout="constrained",
)
axes.plot(
    pulse_lengths_us,
    signal_strengths,
    linestyle="",
    marker="o",
    markersize=0.8,
)
axes.set_ylabel("Signal strength [a.u.]")
axes.set_xlabel("Pulse length [μs]")

plot.adjust_for_thesis(axes)

fig.savefig("rabi_nutation_raw.pdf")

In [ ]:
p  # Try to plot simple least squares fit
fit = process.fit_decaying_squared_sinusoid(pulse_lengths_us, signal_strengths)
pulse_lengths_us_fine = np.linspace(
    pulse_lengths_us[0], pulse_lengths_us[-1], len(pulse_lengths_us) * 10
)
axes.plot(
    pulse_lengths_us_fine,
    fit["function"](pulse_lengths_us_fine),
    linestyle="-",
    linewidth=2,
)

# Save plot
fig.savefig("rabi_nutation_fit.pdf")
fig